In [1]:
# Importar librerías necesarias
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# Descargar recursos necesarios de nltk
nltk.download('punkt')

# Ruta del dataset
ruta_dataset = r'podcastdata_dataset.csv'

# Cargar el dataset
datos = pd.read_csv(ruta_dataset)

# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
print(datos.head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\galot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Primeras filas del dataset:
   id            guest                    title  \
0   1      Max Tegmark                 Life 3.0   
1   2    Christof Koch            Consciousness   
2   3    Steven Pinker  AI in the Age of Reason   
3   4    Yoshua Bengio            Deep Learning   
4   5  Vladimir Vapnik     Statistical Learning   

                                                text  
0  As part of MIT course 6S099, Artificial Genera...  
1  As part of MIT course 6S099 on artificial gene...  
2  You've studied the human mind, cognition, lang...  
3  What difference between biological neural netw...  
4  The following is a conversation with Vladimir ...  


In [2]:
# Limpiar espacios iniciales y finales en los títulos de los episodios
datos['title'] = datos['title'].str.strip()

# Contar títulos únicos
titulos_unicos = datos['title'].nunique()
print(f"Número de títulos únicos después de la limpieza: {titulos_unicos}")


Número de títulos únicos después de la limpieza: 317


In [3]:
# Agregar una columna con el conteo de palabras en la columna 'text'
datos['conteo_palabras'] = datos['text'].apply(lambda texto: len(str(texto).split()))

# Mostrar el conteo de palabras para los primeros episodios
print("Conteo de palabras por episodio:")
print(datos[['id', 'title', 'conteo_palabras']].head())


Conteo de palabras por episodio:
   id                    title  conteo_palabras
0   1                 Life 3.0            13424
1   2            Consciousness            10217
2   3  AI in the Age of Reason             5989
3   4            Deep Learning             5993
4   5     Statistical Learning             6374


In [4]:
# Contar oraciones usando el tokenizador de oraciones de nltk
datos['conteo_oraciones'] = datos['text'].apply(lambda texto: len(sent_tokenize(str(texto))))

# Mostrar el conteo de oraciones para los primeros episodios
print("Conteo de oraciones por episodio:")
print(datos[['id', 'title', 'conteo_oraciones']].head())


Conteo de oraciones por episodio:
   id                    title  conteo_oraciones
0   1                 Life 3.0               737
1   2            Consciousness               603
2   3  AI in the Age of Reason               325
3   4            Deep Learning               359
4   5     Statistical Learning               551


In [5]:
# Solicitar al usuario el ID del episodio que desea analizar
episodio_id = 1  # Cambiar por el ID deseado

# Filtrar el episodio seleccionado
episodio = datos[datos['id'] == episodio_id]

if episodio.empty:
    print(f"No se encontró el episodio con ID {episodio_id}.")
else:
    # Obtener detalles del episodio
    titulo = episodio['title'].iloc[0]
    conteo_palabras = episodio['conteo_palabras'].iloc[0]
    conteo_oraciones = episodio['conteo_oraciones'].iloc[0]
    
    # Mostrar los detalles del episodio
    print(f"Detalles del episodio {episodio_id}:")
    print(f"Título: {titulo}")
    print(f"Palabras: {conteo_palabras}")
    print(f"Oraciones: {conteo_oraciones}")


Detalles del episodio 1:
Título: Life 3.0
Palabras: 13424
Oraciones: 737


In [6]:
# Procesar episodios y dividir cada texto en oraciones
datos_procesados = []

for _, fila in datos.iterrows():
    ep_id = fila['id']
    texto = str(fila['text'])
    oraciones = sent_tokenize(texto)  # Dividir texto en oraciones
    
    for st_id, oracion in enumerate(oraciones, start=1):
        datos_procesados.append({
            'ep_id': ep_id,
            'st_id': st_id,
            'oracion': oracion,
            'conteo_palabras': len(oracion.split())
        })

# Crear un nuevo DataFrame con las oraciones procesadas
df_oraciones = pd.DataFrame(datos_procesados)

# Mostrar las primeras filas del DataFrame procesado
print("Primeras filas del DataFrame de oraciones:")
print(df_oraciones.head())


Primeras filas del DataFrame de oraciones:
   ep_id  st_id                                            oracion  \
0      1      1  As part of MIT course 6S099, Artificial Genera...   
1      1      2                     He is a professor here at MIT.   
2      1      3  He's a physicist, spent a large part of his ca...   
3      1      4  But he's also studied and delved into the bene...   
4      1      5  Amongst many other things, he is the cofounder...   

   conteo_palabras  
0               19  
1                7  
2               17  
3               17  
4               24  


In [7]:
pip install pandas nltk sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tf-keras


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Generar embeddings para todas las oraciones en bloques (batches)
print("Generando embeddings en bloques. Esto será más rápido...")

# Convertir las oraciones a una lista
oraciones = df_oraciones['oracion'].tolist()

# Generar embeddings en lotes
embeddings = modelo_embeddings.encode(oraciones, batch_size=32, show_progress_bar=True)

# Asignar los embeddings al DataFrame
df_oraciones['embedding'] = embeddings.tolist()

print("Embeddings generados exitosamente.")


In [ ]:
# Importar la librería de SentenceTransformers
from sentence_transformers import SentenceTransformer

# Cargar el modelo preentrenado
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para cada oración (puede tardar un poco)
print("Generando embeddings para las oraciones...")
df_oraciones['embedding'] = df_oraciones['oracion'].apply(lambda texto: modelo_embeddings.encode(texto).tolist())

# Mostrar una muestra del DataFrame con embeddings generados
print("Primeras filas con embeddings generados:")
print(df_oraciones.head())


c:\Users\galot\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\galot\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\galot\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generando embeddings para las oraciones...


In [ ]:
# Guardar el DataFrame procesado con embeddings en un archivo CSV
ruta_salida = r'oraciones_procesadas.csv'
df_oraciones.to_csv(ruta_salida, index=False)

print(f"Los resultados se han guardado en el archivo: {ruta_salida}")
